In [2]:
import re

In [3]:
slice("ABCD",2,4)

slice('ABCD', 2, 4)

In [4]:
"abc"

'abc'

In [5]:
def parse_frams_geno(geno:str, in_paranthesis=False):
    if len(geno) == 0:
        return ["empty"]
    j = geno.find(",")
    if j != -1 and in_paranthesis:
        return ["comma"] + parse_frams_geno(geno[:j]) + parse_frams_geno(geno[j+1:],True)
    i = geno.find("(")
    if i != -1:
        l = geno.rfind(")")
        return ["add_branch"] + parse_frams_geno(geno[:i]) + parse_frams_geno(geno[i+1:l],True)

    if len(geno) == 1:
        return [geno[0]]

    if geno[1] == "[": #handle
        match = re.search(r'\[[^\[\]]*?\](?!\[)', geno)
        type, rest = parse_neuron(match.group()[1:-1])
        return ["add_neuron"] + parse_frams_geno(geno[:match.start()]) + [type] + rest + parse_frams_geno(geno[match.end():])
    return ["concat", geno[0]] + parse_frams_geno(geno[1:])

def parse_neuron(neuron_inside:str | list[str], is_first=True):
    if len(neuron_inside) == 0:
        return ["end"]
    if isinstance(neuron_inside,str):
        neuron_inside = neuron_inside.split(",")
    neuron_type = None
    i = 0
    if ":" not in neuron_inside[0]:
        neuron_type = neuron_inside[0]
        i = 1
    if is_first:
        neuron_type = "N" if not neuron_type else neuron_type
        return neuron_type, parse_neuron(neuron_inside[i:], is_first=False)
    
    first, second = neuron_inside[0].split(":")
    if re.match(r'^-?\d+$',first):
        thing = ["conn", int(first), float(second)]
    else:
        thing = ["prop", first, float(second)]

    return thing + parse_neuron(neuron_inside[1:], is_first=False)


In [ ]:
"abs"[2:]

's'

In [7]:
def simple_parser(geno:str, in_paranthesis=False):
    if len(geno) == 0:
        return ["end"]
    if in_paranthesis:
        depth = 0
        j = -1
        for i, c in enumerate(geno):
            if c == "(" or c == "[":
                depth += 1
            if c == ")" or c == "]":
                depth -= 1
            if c == "," and depth == 0:
                j = i
                break
        if j != -1:
            return ["comma"] + simple_parser(geno[:j]) + simple_parser(geno[j+1:],True)
    if geno[0] == "(":
        return ["branch"] + simple_parser(geno[1:-1],True)

    if geno[0] == "[": #handle
        #match = re.search(r'\[[^\[\]]*?\](?!\[)', geno)
        i = geno.find("]")
        type, proplist = parse_neuron(geno[1:i])
        return ["neuron"] + [type] + proplist + simple_parser(geno[i+1:])
    return [geno[0]] + simple_parser(geno[1:])

In [10]:
from deap import gp
?gp.PrimitiveTree

Init signature: gp.PrimitiveTree(content)
Docstring:     
Tree specifically formatted for optimization of genetic programming
operations. The tree is represented with a list, where the nodes are
appended, or are assumed to have been appended when initializing an object
of this class with a list of primitives and terminals e.g. generated with
the method **gp.generate**, in a depth-first order.
The nodes appended to the tree are required to have an attribute *arity*,
which defines the arity of the primitive. An arity of 0 is expected from
terminals nodes.
File:           d:\studia_2\f1-gp-gomea\.venv\lib\site-packages\deap\gp.py
Type:           type
Subclasses:     

In [ ]:
parse_neuron("N,3:2,fi:2")

('N', ['conn', 3, 2.0, 'prop', 'fi', 2.0, 'end'])

In [15]:
simple_parser("(X(X,,,,(X,X,XX(X))))")

['branch',
 'X',
 'branch',
 'comma',
 'X',
 'end',
 'comma',
 'end',
 'comma',
 'end',
 'comma',
 'end',
 'branch',
 'comma',
 'X',
 'end',
 'comma',
 'X',
 'end',
 'X',
 'X',
 'branch',
 'X',
 'end']

In [13]:
tree = gp.PrimitiveTree(simple_parser("(X(X,,,,(X,X,XX(X))))"))

In [14]:
%matplotlib inline
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.pyplot as plt
import networkx as nx

nodes, edges, labels = gp.graph(tree)
g = nx.Graph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
pos = graphviz_layout(g, prog="dot")

nx.draw_networkx_nodes(g, pos)
nx.draw_networkx_edges(g, pos)
nx.draw_networkx_labels(g, pos, labels)
plt.show()

AttributeError: 'str' object has no attribute 'value'

In [2]:
from functools import partial
def add(a,b,c):
    return a+b+c

partial(add,a=2,b=4)(5)

TypeError: add() got multiple values for argument 'a'

In [ ]:
import re

In [ ]:
"abc"[:2]

'ab'

In [ ]:
import re
geno = "[abc][def]dsa[ghi]"
match = re.search(r'\[[^\[\]]*?\](?!\[)', geno)
match.start()
match.end()

10

In [ ]:
"ABCD"[:2]

'AB'

In [ ]:
#geno = "cccRRRXXX[N][N]X(X,,X(X,,cXCX,R(RRX)),XX)"
geno = "(cccXX[0:2]XX[*][Thr,lo:2](X,,,,(X,X,XX(X))))"
parsed = simple_parser(geno)
print(parsed)

['branch', 'c', 'c', 'c', 'X', 'X', 'neuron', 'N', 'conn', 0, 2.0, 'end', 'X', 'X', 'neuron', '*', 'end', 'neuron', 'Thr', 'prop', 'lo', 2.0, 'end', 'branch', 'comma', 'X', 'end', 'comma', 'end', 'comma', 'end', 'comma', 'end', 'branch', 'comma', 'X', 'end', 'comma', 'X', 'end', 'X', 'X', 'branch', 'X', 'end']


In [ ]:
print(parsed)

['c', 'c', 'c', 'X', 'X', 'neuron', 'N', 'conn', 0, 2.0, 'end', 'X', 'X', 'neuron', '*', 'end', 'neuron', 'Thr', 'prop', 'lo', 2.0, 'end', 'branch', 'comma', 'X', 'end', 'comma', 'end', 'comma', 'end', 'comma', 'end', 'comma', 'branch', 'end', 'comma', 'X', 'end', 'X', 'X', 'branch', 'X', ')', 'end']


In [ ]:
from enum import Enum, auto

class AutoName(Enum):
    @staticmethod
    def _generate_next_value_(name, start, count, last_values):
        return name

class test(AutoName):
    hey = auto()

class test2(Enum):
    skibi = auto()

In [ ]:
list(test) + list(test2)

[<test.hey: 'hey'>, <test2.skibi: 1>]

In [ ]:
test.hey.value

'hey'

In [ ]:
len(geno), len(parsed)

(45, 45)

In [ ]:
from deap import gp
from functools import partial

In [ ]:
gp.MetaEphemeral("nint",partial(lambda s: s,5))

deap.gp.nint